In [1]:
import os
from scipy.io import wavfile
from pydub import AudioSegment
from pydub.silence import split_on_silence

import glob

In [35]:
import os
from tqdm import tqdm
from scipy.io import wavfile
from pydub import AudioSegment
from pydub.silence import split_on_silence

import glob

def split(sound):
    dBFS = sound.dBFS
    chunks = split_on_silence(sound,
        min_silence_len = 100,
        silence_thresh = dBFS-16,
        keep_silence = 100
    )
    return chunks

def combine(_src):
    audio = AudioSegment.empty()
    for i,filename in enumerate(os.listdir(_src)):
        if filename.endswith('.mp3'):
            filename = os.path.join(_src, filename)
            audio += AudioSegment.from_mp3(filename)
    return audio

def save_chunks(chunks, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    counter = 0

    target_length = 5 * 1000
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
        if len(output_chunks[-1]) < target_length:
            output_chunks[-1] += chunk
        else:
            # if the last output chunk is longer than the target length,
            # we can start a new one
            output_chunks.append(chunk)

    for chunk in output_chunks:
        chunk = chunk.set_frame_rate(24000)
        chunk = chunk.set_channels(1)
        counter = counter + 1
        chunk.export(os.path.join(directory, str(counter) + '.wav'), format="wav")

In [10]:
flist = glob.glob('data/spot/*/*')
artists = list(set([f.split('/')[2].split(' - ')[0] for f in flist]))

for k in artists:
    path = 'data/spot_vocals/'+k+'/'
    os.makedirs(path, exist_ok=True)

In [25]:
flist = glob.glob('data/spot/*/*')
vocal_list = []
for f in flist:
    if 'no_vocals.mp3' in f:
        continue
    else:
        vocal_list.append(f)

In [30]:
import shutil

for v in vocal_list:
    artist = v.split('/')[2].split(' - ')[0]
    name = v.split('/')[2].split(' - ')[-1]
    path = 'data/spot_vocals/'+artist+'/'+name+'.mp3'
    shutil.move(v, path)

In [31]:
artists

['Lilas Ikuta',
 'Sakura Fujiwara',
 'Masaharu Fukuyama',
 'Ken Hirai',
 'Ai Otsuka',
 '高野洸',
 'ASIAN KUNG-FU GENERATION',
 'Fujii Kaze',
 'Motohiro Hata',
 'LiSA',
 'My Hair is Bad',
 'King Gnu',
 'RADWIMPS',
 'YUI',
 'Naotaro Moriyama',
 'Sakurako Ohara',
 'Gen Hoshino',
 'Sheena Ringo',
 'Aimyon',
 'aiko']

In [37]:
# output path
__OUTPATH__ = "./data/spot_original"
for p in tqdm(artists):
    directory = './data/spot_vocals/' + p
    save_directory = __OUTPATH__ + '/' + p
    os.makedirs(directory, exist_ok=True)
    audio = combine(directory)
    chunks = split(audio)
    save_chunks(chunks, save_directory)

100%|██████████| 20/20 [06:30<00:00, 19.52s/it]


In [38]:
# 平井堅-114: ボーカル以外の音が入っている

In [39]:
# get all speakers

data_list = glob.glob('./data/spot_original/*/*')
                
import pandas as pd

data_list = pd.DataFrame(data_list)
data_list = data_list.sample(frac=1)

import random

split_idx = round(len(data_list) * 0.1)

test_data = data_list[:split_idx][0].to_list()
train_data = data_list[split_idx:][0].to_list()

In [40]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder().fit(artists)

In [43]:
# write to file 

file_str = ""
for k in train_data:
    artist = k.split('/')[3]
    artist_le = le.transform([artist])[0]
    file_str += '.'+ k + "|" + str(artist_le)+ '\n'
text_file = open(__OUTPATH__ + "/train_list.txt", "w")
text_file.write(file_str)
text_file.close()

file_str = ""
for k in test_data:
    artist = k.split('/')[3]
    artist_le = le.transform([artist])[0]
    file_str += '.'+ k + "|" + str(artist_le)+ '\n'
text_file = open(__OUTPATH__ + "/val_list.txt", "w")
text_file.write(file_str)
text_file.close()